In [167]:
from langchain.agents.structured_output import ToolStrategy
from langchain_community.tools import DuckDuckGoSearchResults
from langchain.agents.middleware import HumanInTheLoopMiddleware
from langchain_groq import ChatGroq
from langgraph.types import Command
from langchain_core.tools import Tool
from langgraph.checkpoint.memory import MemorySaver
from langchain.agents import create_agent
from pydantic import BaseModel, Field
from typing import List, Optional
from typing import Dict, List, Any, Optional
import os
from dotenv import load_dotenv
load_dotenv()

True

In [168]:
ddg_search = DuckDuckGoSearchResults()

ddg_search_tool = Tool(
    name="DuckDuckGoSearch",
    func=ddg_search.run,  
    description=(
        "Use this tool to perform a DuckDuckGo web search and return JSON-formatted results. "
        "Input: a search query string; Output: a JSON array of search results."
    )
)

In [169]:
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [170]:
model_name = "qwen/qwen3-32b"
 
#"moonshotai/kimi-k2-instruct" "qwen/qwen3-32b" "openai/gpt-oss-20b"

model = ChatGroq(
    model_name=model_name,
    temperature=0.8,
    max_tokens=1000,
    max_retries=3,
    seed = 42,
)   


/home/sushovan/sushovan/STUDY/StreamlitApp/strmlit/lib/python3.12/site-packages/pydantic/main.py:250: UserWarning: WARNING! seed is not default parameter.
                    seed was transferred to model_kwargs.
                    Please confirm that seed is what you intended.
  validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)


In [ ]:
class SearchResult(BaseModel):
    """Structured search result"""
    title: str = Field(description="Title of the result")
    url: str = Field(description="URL of the result")
    snippet: str = Field(description="Brief snippet from the result")

class AgentResponse(BaseModel):
    """Structured agent response"""
    answer: str = Field(description="The main answer to the query")
    sources: List[SearchResult] = Field(description="Sources used", default_factory=list)
    confidence: Optional[str] = Field(description="Confidence level", default=None)


In [172]:
# Create the agent
memory = MemorySaver()

In [ ]:
hitl_middleware = HumanInTheLoopMiddleware(
    interrupt_on={
        "DuckDuckGoSearch": True,  #Must match tool name
    },
)

In [174]:
# Use the agent
# Create agent with structured model
agent = create_agent(model, 
                    tools=[ddg_search_tool], 
                    checkpointer=memory,
                    middleware=[hitl_middleware],
                    # response_format=ToolStrategy(AgentResponse),
                    system_prompt="You are a helpful assistant.You must use available tools to answer the user's question.")

In [ ]:

# config = {"configurable": {"thread_id": "xyz123"}}

# user_input = {"messages": [{"role": "user", "content": "India vs Australia Last ODI Score"}]}

# result = agent.invoke(user_input, config=config)

In [ ]:
# ============================================
# Interrupt Handling Functions
# ============================================

def extract_search_query(tool_args: Dict[str, Any]) -> str:
   
    if '__arg1' in tool_args:
        return tool_args['__arg1']
    elif 'query' in tool_args:
        return tool_args['query']
    else:
        return str(tool_args)


def display_action_request(idx: int, tool_name: str, tool_args: Dict[str, Any]) -> None:
    
    print(f"{'─'*60}")
    print(f"Action #{idx}")
    print(f"{'─'*60}")
    print(f"Tool Name: {tool_name}")
    print(f"Tool Arguments: {tool_args}")
    print()
    
    search_query = extract_search_query(tool_args)
    print(f"🔍 Search Query: '{search_query}'")
    print()


def get_user_decision(tool_name: str, tool_args: Dict[str, Any]) -> Dict[str, Any]:
    
    decision = input("Your decision (approve/edit/reject): ").strip().lower()
    print()
    
    if decision == "approve":
        print("✅ APPROVED\n")
        return {"type": "approve"}
    
    elif decision == "edit":
        new_query = input("New search query: ").strip()
        
        if new_query:
            new_args = tool_args.copy()
            if '__arg1' in new_args:
                new_args['__arg1'] = new_query
            elif 'query' in new_args:
                new_args['query'] = new_query
            
            print(f"✅ Modified to: '{new_query}'")
            print("⚠️ Note: Edited tool will be auto-approved on next interrupt\n")
            
            return {
                "type": "edit",
                "edited_action": {
                    "name": tool_name,
                    "args": new_args
                }
            }
        else:
            print("⚠️ No query provided, approving original\n")
            return {"type": "approve"}
    
    elif decision == "reject":
        feedback = input("Rejection reason: ").strip()
        print(f"❌ REJECTED\n")
        
        return {
            "type": "reject",
            "feedback": feedback or "Rejected by user"
        }
    
    else:
        print("⚠️ Invalid decision, defaulting to approve\n")
        return {"type": "approve"}


def process_interrupt(interrupts: List[Any], is_subsequent: bool = False) -> List[Dict[str, Any]]:
    
    decisions = []
    
    # Auto-approve subsequent interrupts (after edits)
    if is_subsequent:
        print("⚡ Auto-approving edited tool call (second interrupt detected)\n")
        for interrupt_obj in interrupts:
            action_requests = interrupt_obj.value.get('action_requests', [])
            decisions.extend([{"type": "approve"}] * len(action_requests))
        return decisions
    
    # Process first interrupt - get user input
    for interrupt_obj in interrupts:
        interrupt_data = interrupt_obj.value
        action_requests = interrupt_data.get('action_requests', [])
        
        for idx, action in enumerate(action_requests, 1):
            tool_name = action['name']
            tool_args = action['args']
            
            display_action_request(idx, tool_name, tool_args)
            decision = get_user_decision(tool_name, tool_args)
            decisions.append(decision)
    
    return decisions


def handle_interrupts(agent, initial_result: Dict[str, Any], config: Dict[str, Any]) -> Dict[str, Any]:
    
    result = initial_result
    interrupt_count = 0
    
    while "__interrupt__" in result:
        interrupt_count += 1
        interrupts = result["__interrupt__"]
        
        print(f"\n🛑 TOOL EXECUTION PAUSED - Approval Required (Round #{interrupt_count})\n")
        
        # Process interrupts (auto-approve if subsequent)
        is_subsequent = interrupt_count > 1
        decisions = process_interrupt(interrupts, is_subsequent)
        
        # Resume execution
        print("▶️ Resuming execution...\n")
        result = agent.invoke(
            Command(resume={"decisions": decisions}),
            config=config
        )
    
    return result


def display_final_response(result: Dict[str, Any]) -> None:
    
    if "messages" in result and result["messages"]:
        last_message = result["messages"][-1]
        
        print("\n" + "="*60)
        print("✨ FINAL RESPONSE")
        print("="*60)
        print(f"\n{last_message.content}\n")
    else:
        print("⚠️ No response available")


# ============================================
# Main Execution Function
# ============================================

def run_agent_with_hitl(query: str, thread_id: str) -> Optional[str]:
    
    # Setup
    config = {"configurable": {"thread_id": thread_id}}
    
    # Initial invoke
    result = agent.invoke(
        {"messages": [("user", query)]},
        config
    )
    
    # Handle interrupts if present
    if "__interrupt__" in result:
        result = handle_interrupts(agent, result, config)
        display_final_response(result)
        
        # Return final content
        if "messages" in result and result["messages"]:
            return result["messages"][-1].content
        return None
    
    else:
        print("✅ No interrupts - execution completed")
        if "messages" in result and result["messages"]:
            response = result["messages"][-1].content
            print(f"\n{response}")
            return response
        return None




## Approve

In [ ]:
response = run_agent_with_hitl(
        query="India vs Australia Last ODI Score",
        thread_id="user-123"
    )
    
   


🛑 TOOL EXECUTION PAUSED - Approval Required (Round #1)

────────────────────────────────────────────────────────────
Action #1
────────────────────────────────────────────────────────────
Tool Name: DuckDuckGoSearch
Tool Arguments: {'__arg1': 'India vs Australia last ODI score'}

🔍 Search Query: 'India vs Australia last ODI score'


✅ APPROVED

▶️ Resuming execution...


✨ FINAL RESPONSE

The most recent ODI between India and Australia concluded with **India winning by 9 wickets** in the 3rd ODI at Sydney. Key highlights include:

- **Rohit Sharma** scoring his **33rd ODI century**.
- **Virat Kohli** contributing with his **75th ODI fifty**.
- A strong **unbroken 168-run partnership** between Rohit and Kohli steering India to victory.

For detailed scorecards and match reports, you can refer to [FirstPost's coverage](https://www.firstpost.com/firstcricket/india-vs-australia-3rd-odi-live-score-ind-vs-aus-full-scorecard-sydney-cricket-ground-kohli-liveblog-13944886.html). 

Note: If the

## Edit

In [ ]:
response = run_agent_with_hitl(
        query="India vs Australia Last ODI Score",
        thread_id="user-124"
    )
    
   


🛑 TOOL EXECUTION PAUSED - Approval Required (Round #1)

────────────────────────────────────────────────────────────
Action #1
────────────────────────────────────────────────────────────
Tool Name: DuckDuckGoSearch
Tool Arguments: {'__arg1': 'India vs Australia Last ODI Score'}

🔍 Search Query: 'India vs Australia Last ODI Score'


✅ Modified to: 'Ind vs Aus 2nd ODI results'
⚠️ Note: Edited tool will be auto-approved on next interrupt

▶️ Resuming execution...


✨ FINAL RESPONSE

In the 2nd ODI between **Australia and India** on **October 23, 2025**, at the **Adelaide Oval**, **Australia won by 1 run** in a thrilling encounter. Here are the key details:

- **Australia's Score**: 265/8 (46.2 overs)  
  - **Cooper Connolly**: 61 runs (53 balls)  
- **India's Score**: 264/9 (50 overs)  
  - **Adam Zampa**: 0 runs (1 over)  

The match was closely contested, with Australia just edging out India in a tight finish. For full scorecards and highlights, you can refer to [ESPNcricinfo](https:/

## Reject

In [ ]:

response = run_agent_with_hitl(
        query="India vs Australia Last ODI Score",
        thread_id="user-125"
    )
    
   


🛑 TOOL EXECUTION PAUSED - Approval Required (Round #1)

────────────────────────────────────────────────────────────
Action #1
────────────────────────────────────────────────────────────
Tool Name: DuckDuckGoSearch
Tool Arguments: {'__arg1': 'India vs Australia last ODI result'}

🔍 Search Query: 'India vs Australia last ODI result'


❌ REJECTED

▶️ Resuming execution...


✨ FINAL RESPONSE

The last ODI between India and Australia concluded in September 2024 as part of the 3-match series in Hyderabad. India won the final ODI by **50 runs**, securing the series 2-1. For the most up-to-date scores or recent matches, check official sports news websites or the ICC's ODI match schedules. Let me know if you'd like help finding those!



In [ ]:
if "__interrupt__" in result:
    interrupts = result["__interrupt__"]
    # Gives the info about the interrupt
    print(interrupts)

    for interrupt_obj in interrupts:
        interrupt_data = interrupt_obj.value
        action_requests = interrupt_data.get('action_requests', [])
        
        for idx, action in enumerate(action_requests, 1):
            tool_name = action['name']
            tool_args = action['args']
    


def get_user_decision(tool_name,tool_args):
    
    decision = input("Your decision (approve/edit/reject): ").strip().lower()
    
    decisions = []
    default_decision = [{"type": "approve"}]
    
    if decision == "approve":
        decisions.append({"type": "approve"})
        return decisions
    
    elif decision == "edit":
        new_query = input("New search query: ").strip()
        
        if new_query:
            new_args = tool_args.copy()
            if '__arg1' in new_args:
                new_args['__arg1'] = new_query
            elif 'query' in new_args:
                new_args['query'] = new_query
            
            print(f" Modified to: '{new_query}'")
            
            decisions.append({
                "type": "edit",
                "edited_action": {
                    "name": tool_name,
                    "args": new_args
                }
            })
            return decisions
        else:
            return default_decision
    
    elif decision == "reject":
        feedback = input("Rejection reason: ").strip()
        
        decisions.append({
            "type": "reject",
            "feedback": feedback
        })
        return decisions
    
    else:
        return default_decision



## Experiment with Hilt

In [ ]:
# if "__interrupt__" in result:
#     interrupt_count = 0
    
#     while "__interrupt__" in result:
#         interrupt_count += 1
#         interrupts = result["__interrupt__"]
        
#         print(f"\n🛑 TOOL EXECUTION PAUSED - Approval Required (Round #{interrupt_count})\n")
        
#         decisions = []
        
#         # If this is a subsequent interrupt after edit, auto-approve
#         if interrupt_count > 1:
#             print("⚡ Auto-approving edited tool call (second interrupt detected)\n")
#             for interrupt_obj in interrupts:
#                 action_requests = interrupt_obj.value.get('action_requests', [])
#                 decisions.extend([{"type": "approve"}] * len(action_requests))
#         else:
#             # First interrupt - get user input
#             for interrupt_obj in interrupts:
#                 interrupt_data = interrupt_obj.value
#                 action_requests = interrupt_data.get('action_requests', [])
                
#                 for idx, action in enumerate(action_requests, 1):
#                     tool_name = action['name']
#                     tool_args = action['args']
                    
#                     print(f"{'─'*60}")
#                     print(f"Action #{idx}")
#                     print(f"{'─'*60}")
#                     print(f"Tool Name: {tool_name}")
#                     print(f"Tool Arguments: {tool_args}")
#                     print()
                    
#                     if '__arg1' in tool_args:
#                         search_query = tool_args['__arg1']
#                     else:
#                         search_query = str(tool_args)
                    
#                     print(f"🔍 Search Query: '{search_query}'")
#                     print()
                    
#                     decision = input("Your decision (approve/edit/reject): ").strip().lower()
#                     print()
                    
#                     if decision == "approve":
#                         print("✅ APPROVED\n")
#                         decisions.append({"type": "approve"})
                        
#                     elif decision == "edit":
#                         new_query = input(f"New search query: ").strip()
#                         if new_query:
#                             new_args = tool_args.copy()
#                             if '__arg1' in new_args:
#                                 new_args['__arg1'] = new_query
                            
#                             decisions.append({
#                                 "type": "edit",
#                                 "edited_action": {
#                                     "name": tool_name,
#                                     "args": new_args
#                                 }
#                             })
#                             print(f"✅ Modified to: '{new_query}'")
#                             print("⚠️ Note: Edited tool will be auto-approved on next interrupt\n")
#                         else:
#                             decisions.append({"type": "approve"})
                            
#                     elif decision == "reject":
#                         feedback = input("Rejection reason: ").strip()
#                         decisions.append({
#                             "type": "reject",
#                             "feedback": feedback or "Rejected by user"
#                         })
#                         print(f"❌ REJECTED\n")
#                     else:
#                         decisions.append({"type": "approve"})
        
#         print("▶️ Resuming execution...\n")
        
#         result = agent.invoke(
#             Command(resume={"decisions": decisions}),
#             config=config
#         )
    
#     # Extract plain text response
#     if "messages" in result and result["messages"]:
#         last_message = result["messages"][-1]
        
#         print("\n" + "="*60)
#         print("✨ FINAL RESPONSE")
#         print("="*60)
#         print(f"\n{last_message.content}\n")
#     else:
#         print("⚠️ No response available")

# else:
#     print("✅ No interrupts - execution completed")
#     if "messages" in result and result["messages"]:
#         print(f"\n{result['messages'][-1].content}")